- Redhat identity management
- FreeIPA (Identity, Policy, Audit)
- NTP, DNS, LDAP, Kerberos, certs
- ipa-server-install
- need to have valid kerberos ticket for ipa

- https://www.freeipa.org/page/Quick_Start_Guide
- https://www.certdepot.net/rhel7-configure-freeipa-server/

- to create user accounts:
```
kinit admin
ipa user-add lisa
ipa passwd lisa
ipa user-find lisa
```

- authcofig tool to configure authentication

- Steps:

```
yum -y install ipa-server bind-dyndb-ldap
ipa-server-install
systemctl restart sshd # to obtain Kerberos credentials
kinit admin  # verify kerberos authentication
ipa user-find admin # verify ipa access
```

#### Kerberos tickets

to be able to manage IdM Server, you need to log in to IdM Domain to get a Kerberos ticket for the admin

- IMPORTANT, before doing anything, generate ticket: **```kinit admin```**
- show ticket validity: **```klist```**

Authconfig

- configuration files
```
/etc/sysconfig/authconfig
/etc/sssd/sssd.conf
/etc/krb5.cfg
/etc/nsswitch.conf
/etc/nslcd.conf  # old LDAP configuration file, now use /etc/sssd/sssd.conf instead
```

Exercise authconfig

- set up server2 as an IdM server

```
ssh server2

systemctl stop firewall # rembmer to start after
cat > /etc/hosts <<EOF
192.168.4.101 server1.example.com server1
192.168.4.102 server2.example.com server2 
EOF

yum -y install ipa-server bind nds-ldap bind-dyndb-ldap

ipa-server-install
    > yes to configure BIND
    > server2.example.com as host name
    > DNS forwarder yes
    > DNS forwarder: 8.8.8.8

systemclt restart sshd
knit admin
ip user-find admin
```

- create two users: lisa and lori, with the password: password

```
ipa user-add lisa
ipa passwd lisa
ipa user-find lisa
```

- configure your server as a Kerberos client to itself

```
ssh server1
cat > /etc/hosts <<EOF
192.168.4.101 server1.example.com server1
192.168.4.102 server2.example.com server2 
EOF

authconfig-tui
    > Use LDAP
    > Use Kerberos Authentication
    > ldap://server2.example.com
    > Kerberos settings: Realm: EXAMPLE.COM
    > use KDC to locate realm 

```

- test logging in as one of the use

## iSCSI

iSCSI Configuration (Internet SCSI, Internet Small Computer Systems Interface)

- it provides block-level access to storage devices over a TCP/IP network
- iSCSI can be used to transmit data over local area networks (LANs), wide area networks (WANs), or the Internet and can enable location-independent data storage and retrieval
- much cheaper than fibre chanel, it uses ethernet cable
- it is a storage area network (SAN) protocol

#### Terminology

- **IQN**: iSCSI Qualified Name, to identify targets and initiators, it can be used in security
- Initiator: iSCSI client, identified by an IQN
- Target: the service on an iSCSI server that gives access to the backend services (SAN)
- ACL: Access Control List
- discovery: process where initiator find the targets configured on portal
- Portal: ip address and port use by target
- **LUN**: logical unit, the block devices shared through the target
- **TPG**: target portal group, collection of IP addresses and tcp ports to which iscsi target will listen

In [5]:
! cat /proc/partitions

major minor  #blocks  name

   8        0  234431064 sda
   8        1     204800 sda1
   8        2     256000 sda2
   8        3   61747200 sda3
   8        4  172222023 sda4
  11        0    1048575 sr0
 253        0   61745152 dm-0
 253        1   26214400 dm-1
 253        2    4063232 dm-2
 253        3   67108864 dm-3
 253        4   73400320 dm-4
 253        5   31457280 dm-5


In [6]:
# yum -y install lsscsi
!lsscsi

[0:0:0:0]    disk    ATA      ADATA SP550      3B5a  /dev/sda 
[1:0:0:0]    cd/dvd  HL-DT-ST DVD+-RW GU90N    A1B0  /dev/sr0 


#### configurations

- to install iscsi tools: ```yum install -y targetcli```

- commands to create new block, iscsi SAN (on server2)

```
targetcli
    > ls
    > cd backstores
    > block/ create block1 /dev/vgsan/lvsan1 # it creates block storage object block1 using /dev/vgsan/lvsan1
    > ls

    > cd iscsi
    > create iqn.2016-09.com.example.com:target1  # create target

    # to create acls
    > cd iqn.2016-09.com.example.com:target1
    > cd tpg1
    > acls create iqn.2016-09.com.example.com:server2

    # to create luns
    > luns/ create /backstores/block/block1

    # to create portal
    > portals/ create IPADDRESS

# it saves configuration to file /etc/target/saveconfig.json

# open firewall 3260 tcp

firewall-cmd --add-port=3260/tcp --permanent
fiewall-cmd --reload
systemctl enable target
systemctl start target
```

- commands to create iscsi Initiator (client, on server1)

```
yum install -y iscsi-initiator-utils

# set initiator name
vim /etc/iscsi/initiatorname.iscsi

# to find examples
man iscsiadm

iscsiadm --mode discoverydb --type sendtargets --portal TARGET_IPADDR --discover

iscsiadm --mode node --targetname TARGET_IQN --portal TARGET_IPADDR --login

yum -y install lsscsi
lsscsi  - will show new devices

# should show portal information file
ls /var/lib/iscsi/nodes

# important to logout
iscsiadm --mode node --targetname TARGET_IQN --portal TARGET_IPADDR --logout
```

- verifying the iscsi connection

```
# on client
iscsiadm -m session -P 3
iscsiadm -m node -P 3
iscsiadm -m discovery -P 3

# on target
targetcli
```

- Examples in doc: ```man targetcli```

- **IMPORTANT: to mount network device use ```_netdev``` option in fstab**